In [1]:
import logging

import sqlalchemy_dao
from sqlalchemy_dao import Dao

from SmartAnno.utils.ConfigReader import ConfigReader
from SmartAnno.db.ORMs import Filter
from SmartAnno.gui.Workflow import Workflow
from SmartAnno.utils.AnnotationTypeDef import AnnotationTypeDef
from SmartAnno.utils.IntroStep import IntroStep
from SmartAnno.utils.KeywordsFiltering import KeywordsFiltering
from SmartAnno.utils.KeywordsUMLSExtender import KeywordsUMLSExtender
from SmartAnno.utils.KeywordsUMLSExtenderSetup import KeywordsUMLSExtenderSetup

logging.getLogger().setLevel(logging.DEBUG)

ConfigReader('../conf/smartanno_conf.json')

wf = Workflow(config_file=ConfigReader.config_file)
wf.api_key = ConfigReader.getValue("api_key")
wf.dao = Dao('sqlite+pysqlite:///../data/test.sqlite', sqlalchemy_dao.POOL_DISABLED)
wf.task_name = 'language'
wf.append(AnnotationTypeDef(
    '<h3>Annotation types:</h3><p>List all the types you want to identify below. Each type per line.<br/>If you'
    'have too many types, try set up them separately, so that you won&apos;t need to choose from a long list '
    'for each sample. </p>', name='types'))
wf.append(KeywordsFiltering(
    name='keywords'))
wf.append(KeywordsUMLSExtenderSetup(name='umls_extender_setup'))
wf.append(KeywordsUMLSExtender(name='umls_extender', sources=ConfigReader.getValue("umls/sources"),
                               filter_by_length=ConfigReader.getValue("umls/filter_by_length"),
                               filter_by_contains=ConfigReader.getValue("umls/filter_by_contains"),
                               max_query=ConfigReader.getValue("umls/max_query")))
wf.append(
    IntroStep('<h2>Welcome to SmartAnno!</h2><h4>First, let&apos;s import txt data from a directory. </h4>',
              name='intro'))
wf.start()

wf.steps[0].complete()

with wf.dao.create_session() as session:
    records = session.query(Filter).filter(Filter.task_id == wf.task_id) \
        .filter(Filter.type_name == 'Eng')
    record = records.first()
    record.keyword = 'Eng\nEnglish'

wf.steps[1].complete()
wf.steps[2].to_umls_ext_filters['Eng'].value = ('English',)
wf.steps[2].complete()

VBox(children=(HTML(value="<h4>Synonyms of keyword '<b>English</b>'</h4><p>Choose the ones that you want to include in keyword filters:</p>"), Label(value='', layout=Layout(height='5px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), ToggleButtonsMultiSelectionInBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, description='Walnut Nut', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Walnut Nut'), ToggleButton(value=False, description='Plantago lanceolata', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Plantago lanceolata'), ToggleButton(value=False, description='Quercus robur', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Quercus robur'))), HBox(children=(ToggleButton(value=False, description='Plantago lanceolata specific IgE antibody measurement', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Plantago lanceolata specific IgE antibody measurement'), ToggleButton(value=False, description='Taxus baccata', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Taxus baccata'), ToggleButton(value=False, description='Taxus baccata poisoning', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Taxus baccata poisoning'))), HBox(children=(ToggleButton(value=False, description='Parophrys vetulus', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Parophrys vetulus'), ToggleButton(value=False, description='Plantago lanceolata specific immunoglobulin E', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Plantago lanceolata specific immunoglobulin E'), ToggleButton(value=False, description='Hedera helix', layout=Layout(margin='2', min_width='160px', width='100%'), tooltip='Hedera helix')))), layout=Layout(display='flex')),), _dom_classes=('btn-group',)), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Complete', layout=Layout(left='100px', width='90px'), style=ButtonStyle())), layout=Layout(left='10%', width='80%'))), layout=Layout(width='100%'))